In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
!pip install catboost
import catboost
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
df=pd.read_csv('/content/train.csv',na_values=['?','-999','Error','xxxxxxxx'])
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,NaN,Gift Vouchers/Coupons,NaN,Wi-Fi,16:08:02,17.0,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,NaN,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16.0,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14.0,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11.0,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,NaN,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20.0,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [ ]:
df.isnull().sum()

customer_id                         0
Name                                0
age                                 0
gender                              0
security_no                         0
region_category                  5428
membership_category                 0
joining_date                        0
joined_through_referral          5438
referral_id                     17846
preferred_offer_types             288
medium_of_operation              5393
internet_option                     0
last_visit_time                     0
days_since_last_login            1999
avg_time_spent                      0
avg_transaction_value               0
avg_frequency_login_days         3522
points_in_wallet                 3443
used_special_discount               0
offer_application_preference        0
past_complaint                      0
complaint_status                    0
feedback                            0
churn_risk_score                    0
dtype: int64

In [ ]:
df.shape

(36992, 25)

In [ ]:
df=df.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)

In [ ]:
df['churn_risk_score'].value_counts()

 3    10424
 4    10185
 5     9827
 2     2741
 1     2652
-1     1163
Name: churn_risk_score, dtype: int64

In [ ]:
df.isnull().sum()

age                                0
gender                             0
region_category                 5428
membership_category                0
joining_date                       0
joined_through_referral         5438
preferred_offer_types            288
medium_of_operation             5393
internet_option                    0
days_since_last_login           1999
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days        3522
points_in_wallet                3443
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
churn_risk_score                   0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           36992 non-null  int64  
 1   gender                        36992 non-null  object 
 2   region_category               36992 non-null  object 
 3   membership_category           36992 non-null  object 
 4   joining_year                  36992 non-null  int64  
 5   joined_through_referral       36992 non-null  object 
 6   preferred_offer_types         36992 non-null  object 
 7   medium_of_operation           36992 non-null  object 
 8   internet_option               36992 non-null  object 
 9   days_since_last_login         36992 non-null  float64
 10  avg_time_spent                36992 non-null  float64
 11  avg_transaction_value         36992 non-null  float64
 12  avg_frequency_login_days      36992 non-null  float64
 13  p

In [ ]:
for i in df.columns:
  if df[i].dtype=='float64':
    df[i]=df[i].fillna(df[i].mean())
  else:
    df[i]=df[i].fillna(method='ffill')

In [ ]:
df.isnull().sum().sum()

0

In [ ]:
df['joining_date']=df['joining_date'].apply(lambda x:int(x.split('-')[0]))

In [ ]:
df.rename(columns={'joining_date':'joining_year'},inplace=True)

In [ ]:
df1=pd.get_dummies(df,drop_first=True)

In [ ]:
x=df1.drop('churn_risk_score',axis=1)
y=df1['churn_risk_score']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,stratify=y,random_state=42)

In [ ]:
rd=RandomForestClassifier()

In [ ]:
rd.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rdpred=rd.predict(x_test)

In [ ]:
print(classification_report(y_test,rdpred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       233
           1       0.69      0.78      0.73       530
           2       0.74      0.70      0.72       548
           3       0.88      0.91      0.89      2085
           4       0.70      0.56      0.62      2037
           5       0.69      0.88      0.77      1966

    accuracy                           0.75      7399
   macro avg       0.62      0.64      0.62      7399
weighted avg       0.73      0.75      0.73      7399



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(f1_score(y_test,rdpred,average='macro'))

0.622640795683421


In [ ]:
import xgboost

In [ ]:
xgc=xgboost.XGBClassifier()

In [ ]:
xgc.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgc.fit(x,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgcpred=xgc.predict(x_test)

In [ ]:
print(f1_score(y_test,xgcpred,average='macro'))

0.6312701431600242


In [ ]:
params={"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }


In [ ]:
rsearch=RandomizedSearchCV(xgc, param_distributions = params, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rsearch.fit(x,y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 13.6min


KeyboardInterrupt: ignored

In [ ]:
cat=catboost.CatBoostClassifier()

In [ ]:
cat.fit(x_train,y_train)

Learning rate set to 0.093411
0:	learn: 1.6067132	total: 97ms	remaining: 1m 36s
1:	learn: 1.4830459	total: 129ms	remaining: 1m 4s
2:	learn: 1.3837997	total: 161ms	remaining: 53.4s
3:	learn: 1.3062293	total: 192ms	remaining: 47.8s
4:	learn: 1.2375504	total: 232ms	remaining: 46.3s
5:	learn: 1.1823289	total: 279ms	remaining: 46.1s
6:	learn: 1.1370637	total: 309ms	remaining: 43.8s
7:	learn: 1.0842878	total: 338ms	remaining: 41.9s
8:	learn: 1.0351265	total: 368ms	remaining: 40.5s
9:	learn: 0.9954285	total: 402ms	remaining: 39.8s
10:	learn: 0.9573812	total: 437ms	remaining: 39.3s
11:	learn: 0.9245510	total: 469ms	remaining: 38.6s
12:	learn: 0.8931396	total: 500ms	remaining: 38s
13:	learn: 0.8685924	total: 541ms	remaining: 38.1s
14:	learn: 0.8466070	total: 571ms	remaining: 37.5s
15:	learn: 0.8262263	total: 602ms	remaining: 37s
16:	learn: 0.8098421	total: 633ms	remaining: 36.6s
17:	learn: 0.7947982	total: 666ms	remaining: 36.4s
18:	learn: 0.7779865	total: 698ms	remaining: 36s
19:	learn: 0.7623

In [ ]:
catpred=cat.predict(x_test)

In [ ]:
print(f1_score(y_test,catpred,average='macro'))

0.6274702709040306


In [ ]:
import lightgbm

In [ ]:
lgb=lightgbm.LGBMClassifier()

In [ ]:
lgb.fit(x_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
lgbpred=lgb.predict(x_test)

In [ ]:
print(f1_score(y_test,lgbpred,average='macro'))

0.6273492316314816


In [ ]:
test=pd.read_csv('/content/test.csv',na_values=['?','-999','Error','xxxxxxxx'])
test.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,fffe43004900440031003700300030003400,Alethia Meints,50,F,OQJ1XAY,Village,Premium Membership,2015-11-02,No,NaN,Without Offers,Smartphone,Wi-Fi,07:19:30,12.0,386.26,40721.44,7.0,733.830000,Yes,No,No,Not Applicable,Poor Product Quality
1,fffe43004900440031003900370037003300,Ming Lopez,41,M,OUQRPKO,Village,Gold Membership,2016-03-01,No,NaN,Without Offers,Desktop,Fiber_Optic,22:21:16,11.0,37.80,9644.40,9.0,726.000000,Yes,No,No,Not Applicable,Poor Website
2,fffe43004900440034003800360037003000,Carina Flannigan,31,F,02J2RE7,Town,Silver Membership,2017-03-03,No,NaN,Gift Vouchers/Coupons,Both,Mobile_Data,16:40:39,18.0,215.36,3693.25,21.0,713.780000,Yes,No,Yes,Solved in Follow-up,No reason specified
3,fffe43004900440036003200370033003400,Kyung Wanner,64,M,5YEQIF1,Town,Silver Membership,2017-08-18,Yes,CID8941,Credit/Debit Card Offers,NaN,Fiber_Optic,14:56:17,NaN,44.57,36809.56,11.0,744.970000,Yes,No,Yes,No Information Available,Too many ads
4,fffe43004900440035003000370031003900,Enola Gatto,16,F,100RYB5,Town,No Membership,2015-05-05,Yes,CID5690,Without Offers,Smartphone,Mobile_Data,02:57:53,6.0,349.88,40675.86,8.0,299.048351,No,Yes,Yes,Solved in Follow-up,Poor Website


In [ ]:
test=test.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)

In [ ]:
for i in test.columns:
  if test[i].dtype=='float64':
    test[i]=test[i].fillna(test[i].mean())
  else:
    test[i]=test[i].fillna(method='ffill')

In [ ]:
test['joining_date']=test['joining_date'].apply(lambda x:int(x.split('-')[0]))

In [ ]:
test.rename(columns={'joining_date':'joining_year'},inplace=True)

In [ ]:
test1=pd.get_dummies(test,drop_first=True)

In [ ]:
test1

,age,joining_year,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,gender_M,gender_Unknown,region_category_Town,region_category_Village,membership_category_Gold Membership,membership_category_No Membership,membership_category_Platinum Membership,membership_category_Premium Membership,membership_category_Silver Membership,joined_through_referral_Yes,preferred_offer_types_Gift Vouchers/Coupons,preferred_offer_types_Without Offers,medium_of_operation_Desktop,medium_of_operation_Smartphone,internet_option_Mobile_Data,internet_option_Wi-Fi,used_special_discount_Yes,offer_application_preference_Yes,past_complaint_Yes,complaint_status_Not Applicable,complaint_status_Solved,complaint_status_Solved in Follow-up,complaint_status_Unsolved,feedback_Poor Customer Service,feedback_Poor Product Quality,feedback_Poor Website,feedback_Products always in Stock,feedback_Quality Customer Care,feedback_Reasonable Price,feedback_Too many ads,feedback_User Friendly Website
0,50,2015,12.000000,386.26,40721.44,7.000000,733.830000,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
1,41,2016,11.000000,37.80,9644.40,9.000000,726.000000,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
2,31,2017,18.000000,215.36,3693.25,21.000000,713.780000,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
3,64,2017,12.691115,44.57,36809.56,11.000000,744.970000,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
4,16,2015,6.000000,349.88,40675.86,8.000000,299.048351,0,0,1,0,0,1,0,0,0,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19914,12,2015,16.000000,103.57,46279.35,18.000000,708.120000,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
19915,40,2017,21.000000,63.19,23466.26,15.899058,574.340000,0,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
19916,55,2015,18.000000,68.72,17903.75,24.000000,564.300000,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
19917,17,2016,3.000000,119.54,14057.09,22.000000,606.340000,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
rdpred=xgc.predict(test1)

In [ ]:
df2=pd.read_csv('test.csv',usecols=['customer_id'])

In [ ]:
df2['churn_risk_score ']=rdpred

In [ ]:
df2.to_csv('xgboost4.csv')